In [370]:
using Pkg
Pkg.add(PackageSpec(path="https://github.com/diegozea/ROC.jl"))

   Cloning git-repo `https://github.com/diegozea/ROC.jl`
  Updating git-repo `https://github.com/diegozea/ROC.jl`.0 %46.3 %>  ]  93.1 %
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
 Installed Infinity ─ v0.2.3
  Updating `~/.julia/environments/v1.0/Project.toml`
  [e4f92426] + ROC v0.1.0 #master (https://github.com/diegozea/ROC.jl)
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [a303e19e] + Infinity v0.2.3
  [e4f92426] + ROC v0.1.0 #master (https://github.com/diegozea/ROC.jl)


In [391]:
using JuMP, Gurobi, CSV, LinearAlgebra, DataFrames, Random, Distributions, Statistics,MLBase, CPUTime,ScikitLearn ,MLDataUtils
@sk_import metrics: roc_auc_score
gurobi_env = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00007fa9e453c000)

In [429]:
function one_hot_encode(X, names)
    X2 = deepcopy(X)
    select!(X2, Not(Symbol.(names)))
    for i in names
        vales = unique(X[i])
        for j in 1:length(vales)-1
           X2[Symbol(string(i)*"_"*string(vales[j]))] = (X[i].==vales[j])*1
        end
    end
    return X2
end

function normalize(X, names)
    X2 = deepcopy(X)
    select!(X2, Not(Symbol.(names)))
    for j in names
        X2[j] = (X[:,j] .- mean(X[:,j])) / std(X[:,j])
    end
    return X2
end

function clean(X)
    n,p = size(X)
    X2 = deepcopy(X)
    i = 0
    while i < n
        i += 1
        if "?" in X[i,:]
            X = X[1:end .!= i, :]
            i -= 1
        end
        n,_ = size(X)
    end
    return X
end     

function toNum(df, names)
    n,p = size(df)
    for name in names
        if !(isa(df[1,name], Int64) || isa(df[1,name], Float64))
            temp = zeros(n)
            for i=1:n
                temp[i] = parse(Float64,df[i,name])
            end
            df[!,name] = temp
        end
    end
    return df
end

function preprocess(df, categorical_vars, numerical_vars)
    df = clean(df)
    df = toNum(df,numerical_vars)
    df = normalize(df,numerical_vars)
    df = one_hot_encode(df[:,1:end], categorical_vars) 
    df[df[:,end].==0,end] .= -1
    return df
end

preprocess (generic function with 1 method)

In [700]:
### Utils Functions ###
function compute_∇f(w_k, y, X, λ)
    n, p = size(X)
    temp = zeros(p)
    for i in 1:n
         t = min(exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k),100000)
         Δ = (1/(1+t))*t*(-y[i]*Array(X[i,:]) .+ 2*λ*w_k)
#        Δ = (-1/(1+exp(y[i]*dot(w_k,X[i,:])+λ*transpose(w_k)*w_k)))*(y[i].*X[i,:] .+ 2*λ*w_k)
        temp = temp + Δ
#         if i >= n-5
#             println("Δ ", Δ)
#             println(exp(-y[i]*(transpose(w_k)*Array(X[i,:]))+λ*transpose(w_k)*w_k))
# #             for i in 1:5
# #                 println("y ",y[i])
# #                 println("X ",X[i,:])
# #                 println("w ",w_k[:])
# #                 #println(log(1+exp(-y[i]*dot(X[i,:], w_k))))
# #                 println()
# #             end
#         end
#         if i == n
#             println("temp",temp)
#         end
    end
#     for i in 1:5
#         println("y ",y[i])
#         println("X ",X[i,:])
#         println("w ",w_k[:])
#         #println(log(1+exp(-y[i]*dot(X[i,:], w_k))))
#         println()
#     end
#     println("temp",temp)
    ∇f_k = temp
    #println("∇f_k",∇f_k)
    return ∇f_k
end

compute_∇f (generic function with 1 method)

In [701]:
### Cutting Planes Implementation ###
function LR_cutting_planes(y, X, ε, λ)
    errors = []
    n, p = size(X)
    # Initialization values and step 0
    w_0 = [0 for i in 1:p]
    #w_0 = [rand(Uniform(-0.5, 0.5)) for i in 1:p]
    f_0 = sum(log(1+exp(-y[i]*dot(X[i,:], w_0)+λ*transpose(w_0)*w_0)) for i=1:n)
    ∇f_0 = compute_∇f(w_0, y, X, λ)

    # Outer minimization problem
    outer_min_model = Model(solver=GurobiSolver(OutputFlag=0, gurobi_env))
    @variable(outer_min_model, t >= 0)
    @variable(outer_min_model, w[1:p])
    #@constraint(outer_min_model, [j=1:p], -1 <= w[j] <= 1)
    @constraint(outer_min_model, t >= f_0 + (dot(∇f_0, w)-dot(∇f_0, w_0)))
    @constraint(outer_min_model, [j=1:p], 10 >= w[j])
    @constraint(outer_min_model, [j=1:p], w[j] >= -10)
    @objective(outer_min_model, Min, t)
    k = 1 # Number of constraints in the final problem
    solve(outer_min_model)

    # New steps k
    t_k = getvalue(t)
    w_k = getvalue(w)  
    f_k = sum(min(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)),100) for i=1:n)
    ∇f_k = compute_∇f(w_k, y, X, λ)
    
    while abs(f_k - t_k) >= ε # error

        push!(errors, f_k - t_k)
        @constraint(outer_min_model,t >= f_k +(dot(∇f_k, w)-dot(∇f_k, w_k)))
        k += 1
        solve(outer_min_model)
        # Updating all the values
        t_k = getvalue(t)
        w_k = getvalue(w)
        f_k = sum(min(log(1+exp(-y[i]*dot(X[i,:], w_k)+λ*transpose(w_k)*w_k)),1000) for i=1:n)

        ∇f_k = compute_∇f(w_k, y, X, λ)
         if k%500 == 0
             println("Number of constraints: ", k, "\t Error = ", abs(t_k - f_k))
#             println("f",f_k)
#              println("w",w_k)
#              println("∇f_k",∇f_k)
         end
        if k > 20000
            break
        end
    end
    push!(errors, f_k - t_k)
    return t_k, f_k, w_k, errors
end


LR_cutting_planes (generic function with 1 method)

In [702]:
function robust_LG_valid(X, y, ε, lambda_vals; method=LR_cutting_planes, split_at=0.8)
    n,p = size(X)
    split = convert(Int,floor(split_at*n))
    permuted_indices = randperm(n)
    train_indices, valid_indices = permuted_indices[1:split], permuted_indices[split+1:end]
    X_train, y_train = X[train_indices,:], y[train_indices]
    X_valid, y_valid = X[valid_indices,:], y[valid_indices]
    
    accuracies = zeros(length(rho_vals))
    for (i,λ) in enumerate(lambda_vals)
        println(i)
        t, f, w, e = method(y_train, X_train, ε, λ)
        pred = 1 ./ (1 .+ exp.(-(Matrix(X_valid)*w).+λ*transpose(w)*w)) .> 0.5
        accuracies[i] = 1-sum(pred .!= (y_valid .== 1))/length(y_valid)
    end
    IJulia.clear_output()

    i_best = argmax(accuracies)
    t, f, w_best, e = method(y, X, ε,rho_vals[i_best])
    return w_best, lambda_vals[i_best], accuracies
end

robust_LG_valid (generic function with 1 method)

In [703]:
function test(df, categorical_vars, numerical_vars, test_split, validation_split, ε, lambda_vals, seed)
    Random.seed!(seed)
    start = time()
    n,_ = size(df)
    permuted_indices = randperm(n)
    test_split = convert(Int,floor(validation_split*n))
    train_indices, test_indices = permuted_indices[1:test_split], permuted_indices[test_split+1:end]
    train = df[train_indices,:]
    test = df[test_indices,:]
    train_X = train[:,1:end-1]
    train_y = train[:,end]
    test_X = test[:,1:end-1]
    test_y = test[:,end]
    
    IJulia.clear_output()
    println("Enter cross-validation")
    w, λ, errors  = robust_LG_valid(train_X, train_y, ε, lambda_vals; method=LR_cutting_planes, split_at=validation_split)
    elapsed = time() - start
    pred_prob = 1 ./ (1 .+ exp.(-(Matrix(test_X)*w).+λ*transpose(w)*w))
    pred = pred_prob.> 0.5
    accuracy = 1-sum(pred .!= (test_y .== 1))/length(test_y)
    auc = roc_auc_score(test_y ,pred_prob)
    IJulia.clear_output()

    return auc, accuracy, λ, elapsed, w
end

test (generic function with 3 methods)

In [682]:
df = CSV.read("Data/caesarian.csv";header=true)
size(df)

(80, 6)

In [683]:
lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1]
categorical_vars = Symbol.(["Delivery number" ;"Delivery time";"Blood of Pressure";"Heart Problem"])
numerical_vars = Symbol.(["Age"])
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(80, 10)

In [684]:
seed = 1
auc, acc, lambda, elapsed, w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, 1)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7575757575757576 Accuracy: 0.6 λ: 1.0e-5 Time: 2.2049567699432373

In [685]:
w

9-element Array{Float64,1}:
 -1.2491864081369384
 -0.3937010893346649
  0.8600389649568139
 -0.3205883639048049
 -0.8178133645505835
  0.6018132119061007
  1.3387212023583488
  0.416461190330931 
  1.500084987129225 

In [542]:
seed = 2
auc, acc, lambda, elapsed = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7637362637362638 Accuracy: 0.7 λ: 1.0e-5 Time: 1.3622229099273682

In [543]:
seed = 3
auc, acc, lambda, elapsed = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.43499999999999994 Accuracy: 0.44999999999999996 λ: 0.05 Time: 1.4523191452026367

In [544]:
seed = 4
auc, acc, lambda, elapsed = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.609375 Accuracy: 0.5 λ: 1.0e-5 Time: 1.463482141494751

In [692]:
df = CSV.read("Data/monks-1.test";header=false)[:,2:end]
select!(df, Not(Symbol.("Column9")))
df[!, 1], df[!, end] = df[!, end], df[!, 1]
size(df)

(432, 7)

In [693]:
lambda_vals = [0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5]
categorical_vars = propertynames(df)
numerical_vars = []
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(432, 12)

In [694]:
seed = 1
auc, acc, lambda, elapsed,w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7124137931034483 Accuracy: 0.8055555555555556 λ: 0.0001 Time: 14.171710014343262

In [695]:
w

11-element Array{Float64,1}:
 -0.19921007152857848
 -0.3393916056922839 
 -0.23309065923756547
 -0.3933468941032062 
 -0.10828160207035857
  0.12567416466113626
 -0.12158299942954598
 -0.13455729769572078
 20.0                
 -0.11027509123696333
 -0.17074361022605233

In [613]:
seed = 2
auc, acc, lambda, elapsed = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.7596153846153846 Accuracy: 0.7777777777777778 λ: 0.1 Time: 11.854342937469482

In [614]:
w

11-element Array{Float64,1}:
 -0.19967330500412425
 -0.34063384525125345
 -0.23428409783236667
 -0.3926567265808357 
 -0.1083289833942036 
  0.1260091038918896 
 -0.1214634688072341 
 -0.13265833242863104
 10.0                
 -0.11065600270723822
 -0.17023159739032653

In [549]:
seed = 3
auc, acc, lambda, elapsed = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.6411663807890222 Accuracy: 0.7222222222222222 λ: 0.0001 Time: 11.339162111282349

In [704]:
df = CSV.read("Data/credit-screening/crx.data";header=false)
n,p=size(df)

(690, 16)

In [705]:
categorical_vars = propertynames(df[1,vcat(1,4:7,9:10,12:13,16)])
numerical_vars = propertynames(df[1,vcat(2:3,8,11,14:15)])
lambda_vals = [0.0001,0.0005,0.001,0.005,0.01]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(653, 38)

In [706]:
seed = 1
auc, acc, lambda, elapsed,w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9172988249293471 Accuracy: 0.8780487804878049 λ: 0.0001 Time: 332.22172808647156

In [564]:
seed = 2
auc, acc, lambda, elapsed,w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.958049371497804 Accuracy: 0.8841463414634146 λ: 0.005 Time: 467.2927370071411

In [565]:
seed = 3
auc, acc, lambda, elapsed,w = test(df,categorical_vars,numerical_vars,0.75,0.75,0.0001, lambda_vals, seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.917572463768116 Accuracy: 0.8536585365853658 λ: 0.01 Time: 631.3079349994659

In [675]:
df = CSV.read("Data/framingham.csv";header=true)
size(df)

(3658, 16)

In [676]:
categorical_vars = Symbol.(["education"])
numerical_vars = Symbol.(["age" ;"cigsPerDay";"totChol";"sysBP";"diaBP";"BMI";"heartRate";"glucose"])
lambda_vals = [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.2]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(3658, 18)

In [678]:
seed = 1
auc, acc, lambda,elapsed,w = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.8971686633756418 Accuracy: 0.8087431693989071 λ: 0.001 Time: 291.6356430053711

In [578]:
seed = 2
auc, acc, lambda,elapsed = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.8835259112323331 Accuracy: 0.8185792349726776 λ: 1.0e-5 Time: 288.1769390106201

In [579]:
seed = 3
auc, acc, lambda,elapsed = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.0001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

AUC: 0.9008132903545748 Accuracy: 0.8360655737704918 λ: 1.0e-5 Time: 292.8686800003052

In [696]:
df = CSV.read("Data/default of credit card clients.csv";header=true)
n,p = size(df)

(30000, 25)

In [598]:
propertynames(df[1,vcat(4:5)])

2-element Array{Symbol,1}:
 :EDUCATION
 :MARRIAGE 

In [697]:
categorical_vars = propertynames(df[1,vcat(4:5)])
numerical_vars = propertynames(df[1,vcat([2,6],7:24)])
lambda_vals = [0.0001,0.001,0.01]
df = preprocess(df, categorical_vars, numerical_vars)
n,p=size(df)

(30000, 32)

In [698]:
seed = 1
auc, acc, lambda,elapsed = test(df,categorical_vars,numerical_vars, 0.75, 0.75,0.001, lambda_vals,seed)
print("AUC: ",auc," Accuracy: ",acc, " λ: ", lambda, " Time: ", elapsed)

Enter cross-validation
1
Number of constraints: 500	 Error = 6234.1767934946165
w[-0.000631795, 10.3451, -20.0, 6.25596, 0.290806, -1.49541, 0.473608, 3.12466, 3.80476, 0.340599, -6.60304, -20.0, -20.0, 20.0, 20.0, -20.0, 20.0, 0.432675, -9.31761, -20.0, -5.50421, -6.81292, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0]
∇f_k[1.64637e6, 609.756, -42.1734, 501.664, -48.031, -176.793, -158.085, 9.98766, -12.0211, -147.042, -284.085, -150.372, -65.343, 31.1591, 36.586, -9.79421, 14.6661, 300.466, 101.046, -66.9913, -77.6335, -49.8635, -79.8858, 99.5099, 174.891, 14.7967, -3.27591, -2.77123, -3.27656, 218.28, 196.331]
Number of constraints: 1000	 Error = 2885.396220048346
w[0.000142393, 6.73046, -1.07427, -7.97781, 3.0754, -0.822048, -1.88209, 1.10586, -0.672262, -5.78973, 4.95259, 7.80473, -20.0, 20.0, -4.44127, -20.0, 20.0, -1.51711, -20.0, -20.0, 1.93366, -2.6981, 3.43646, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0]
∇f_k[4.90704e6, 545.433, 51.901, -105.673, 2

InterruptException: InterruptException: